In [861]:
import pandas as pd
import sqlite3

#### Подключение к базе данных

In [862]:
try:
    conn = sqlite3.connect('../data/checking-logs.sqlite')
    print("Успешное подключение к базе данных.")
except sqlite3.Error as e:
    print(f"Ошибка при подключении к базе данных: {e}")
    exit()

Успешное подключение к базе данных.


#### Получаем 10 строк таблицы test для проверки

In [863]:
query = "SELECT * FROM deadlines LIMIT 10"
test = pd.io.sql.read_sql(query, conn)
test

,index,labs,deadlines
0,0,laba04,1587945599
1,1,laba04s,1587945599
2,2,laba05,1588550399
3,4,laba06,1590364799
4,5,laba06s,1590364799
5,3,project1,1589673599


#### Минимальное значение дельты между первым коммитом пользователя и крайним сроком соответствующей лабораторной работы

In [864]:
query = """
    SELECT
        t.uid,
        CAST(MIN(JULIANDAY(t.first_commit_ts) - JULIANDAY(d.deadlines, 'unixepoch')) * 24 AS INTEGER) AS min_delta_hours
    FROM test AS t
    JOIN deadlines AS d ON t.labname = d.labs
    WHERE t.labname != 'project1'
    ORDER BY
        min_delta_hours
    LIMIT 1;
"""
df_min = pd.read_sql_query(query, conn)
df_min

,uid,min_delta_hours
0,user_30,-202


#### Максимальное значение дельты между первым коммитом пользователя и крайним сроком соответствующей лабораторной работы

In [865]:
query = """
    SELECT
        t.uid,
        CAST(MAX(JULIANDAY(t.first_commit_ts) - JULIANDAY(d.deadlines, 'unixepoch')) * 24 AS INTEGER) AS max_delta_hours
    FROM test AS t
    JOIN deadlines AS d ON t.labname = d.labs
    WHERE t.labname != 'project1'
    ORDER BY
        max_delta_hours
    LIMIT 1;
"""
df_max = pd.read_sql_query(query, conn)
df_max

,uid,max_delta_hours
0,user_25,-2


#### Среднее значение дельты между первым коммитом пользователя и крайним сроком соответствующей лабораторной работы

In [866]:
query = """
    SELECT
        CAST(AVG(JULIANDAY(t.first_commit_ts) - JULIANDAY(d.deadlines, 'unixepoch')) * 24 AS INTEGER) AS avg_delta_hours
    FROM test AS t
    JOIN deadlines AS d ON t.labname = d.labs
    WHERE t.labname != 'project1'
    ORDER BY
        avg_delta_hours
    LIMIT 1;
"""
df_avg = pd.read_sql_query(query, conn)
df_avg

,avg_delta_hours
0,-89


#### Проверка гипотезы о том, что пользователи, которые посетили новостную ленту всего несколько раз

In [867]:
query = """
    CREATE TABLE IF NOT EXISTS views_diff AS
    SELECT
        t.uid AS uid,
        CAST(AVG(JULIANDAY(t.first_commit_ts) - JULIANDAY(d.deadlines, 'unixepoch')) * 24 AS INTEGER) AS avg_diff,
        COUNT(uid) AS pageviews
    FROM test AS t
    JOIN deadlines AS d ON t.labname = d.labs
    WHERE t.labname != 'project1'
    GROUP BY
        t.uid
"""
cursor = conn.cursor()
cursor.execute(query)
conn.commit()
print("Таблица 'views_diff' создана и заполнена.")

query_select = "SELECT * FROM views_diff"
views_diff = pd.read_sql_query(query_select, conn)
views_diff

Таблица 'views_diff' создана и заполнена.


,uid,avg_diff,pageviews
0,user_1,-65,5
1,user_10,-75,5
2,user_14,-159,3
3,user_17,-62,5
4,user_18,-6,3
5,user_19,-99,4
6,user_21,-96,4
7,user_25,-93,5
8,user_28,-86,5
9,user_3,-105,5


##### Расчет коэффициента корреляции между количеством просмотров страниц и разницей

In [868]:
correlation = views_diff['pageviews'].corr(views_diff['avg_diff'])
print(f"Коэффициент корреляции между pageviews и avg_diff: {correlation}")

Коэффициент корреляции между pageviews и avg_diff: 0.11855414786372757


#### Закрытие соединения

In [869]:
conn.close()